This notebook shows the sequence of commands for an "end-to-end" verification. The idea is to show that we are able to 

    1 - Startup the telescope.
    2 - Acquire calibration data.
    3 - Initialize the observatory for the night.
    4 - Slew to a target.
    5 - Take an image of the target. 
    6 - Read the image from the butler and display it.
    7 - Shutdown the telescope.

The first step in this notebook is to load all the required libraries. 

In [1]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [2]:
import os
import lsst.daf.persistence as dafPersist

In [3]:
import matplotlib
%matplotlib widget
matplotlib.rcParams["figure.figsize"] = (8, 6)
matplotlib.rcParams["font.size"] = 12

In [4]:
import astropy

This next cell defines a method to display the image at the end. 

In [ ]:
def display(image, mask=None, colors=None, alpha=0.40, **kwds):
    box = Box2D(image.getBBox())
    extent = (box.getMinX(), box.getMaxX(), box.getMinY(), box.getMaxY())
    kwds.setdefault("extent", extent)
    kwds.setdefault("origin", "lower")
    kwds.setdefault("interpolation", "nearest")
    matplotlib.pyplot.imshow(image.array, **kwds)
    kwds.pop("vmin", None)
    kwds.pop("vmax", None)
    kwds.pop("norm", None)
    kwds.pop("cmap", None)
    if mask is not None:
        for plane, color in colors.items():
            array = np.zeros(mask.array.shape + (4,), dtype=float)
            rgba = np.array(matplotlib.colors.hex2color(matplotlib.colors.cnames[color]) + (alpha, ),
                            dtype=float)
            np.multiply.outer((mask.array & mask.getPlaneBitMask(plane)).astype(bool), rgba, out=array)
            matplotlib.pyplot.imshow(array, **kwds)

In [ ]:
from lsst.afw.geom import Box2D, Box2I, Point2I, Extent2I

Before proceeding, add a logger and set logging level to `DEBUG` so we can get status message from the various tasks.

In [5]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

Next we initialize two high level classes to command the ATTCS and LATISS.

In [6]:
attcs = ATTCS()

In [7]:
latiss = LATISS(attcs.domain)

Read historical data in 5.97 sec
Read historical data in 6.24 sec
Read historical data in 6.71 sec
Read historical data in 6.88 sec
Read historical data in 7.37 sec
Read historical data in 7.79 sec
Read historical data in 8.66 sec


In [8]:
await asyncio.gather(attcs.start_task,
                     latiss.start_task)

Read historical data in 4.39 sec
Read historical data in 4.57 sec
Read historical data in 4.70 sec
Read historical data in 4.81 sec
RemoteEvent(ATDome, 0, logMessage) falling behind; read 19 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 15 messages


[[None, None, None, None, None, None, None], [None, None, None, None]]

RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 17 messages


Since there is no reason do put these components in standby at a given time, let us assume `LATISS` is already enabled. The same is not true for ATTCS which must be put in standby at the end of the night so the telescope can be left in a safe mode. 

For that we call the `enable` method in ATTCS. This will leave the telescope and its subsystems ready to go. 

Note that the ATHexapod simulator is not working right now. For that we can tell attcs to skip it in enable and startup operations.

In [ ]:
attcs.check.athexapod = False

In [ ]:
await attcs.enable(settings={
    'atmcs': "",
    'atptg': "",
    'ataos': "",
    'atpneumatics': "",
    'athexapod': "Default1",
    'atdome': "",
    'atdometrajectory': ""})

Now we can use LATISS high level method `take_bias` to take a series of bias images.

In [ ]:
await latiss.take_bias(nbias=5)

There is also a method `take_darks` that can be used to take series of dark images.

In [ ]:
await latiss.take_darks(ndarks=5, exptime=15.)

It is also possible to take flat frames. For that, we would have to move the telescope and dome to the flat field position, switch on the lamps and take flat images.

We can position the telescope and dome with the `prepare_for_flatfield` method on the ATTCS. We don't have a simulator for flat lamps now so we will skip it.

In [ ]:
await attcs.prepare_for_flatfield()

In [ ]:
await latiss.take_flats(exptime=5., nflats=5, filter='disperser', grating='ronchi170lpmm')

In [ ]:
await attcs.startup()

Select instrument port. This is not done at the startup.

In [ ]:
await attcs.atmcs.cmd_setInstrumentPort.set_start(port=1)

A trick to aways get a good target to point.

In [ ]:
time = await attcs.atptg.tel_timeAndDate.aget()

In [ ]:
time.lst

In [ ]:
await attcs.slew_icrs(ra=time.lst, dec=-50., slew_timeout=120)

In [ ]:
end_readout = await latiss.take_image(exptime=10., 
                                      shutter=True, 
                                      image_type="OBJECT", 
                                      group_id=astropy.time.Time.now().tai.isot)

In [ ]:
print(end_readout)

Get visit id from the `end_readout` event. 

In [ ]:
visit = int(f"{end_readout.imageDate}{end_readout.imageNumber:05}")
print(visit)

In [ ]:
ingested = await latiss.atarchiver.evt_imageInOODS.next(flush=False, timeout=30)

In [ ]:
print(ingested)

In [ ]:
await asyncio.sleep(15.)

In [ ]:
repo = os.path.join("/mnt/dmcs/oods_butler_repo/repo")
butler = dafPersist.Butler(repo)

In [ ]:
raw = butler.get("raw", expId=visit)

In [ ]:
display(raw.image, cmap=matplotlib.cm.gray, vmin=1e4, vmax=3e4)

Finaly, shutdown the system. 

In [12]:
await attcs.shutdown()

Disabling ATAOS corrections
Disable ATDomeTrajectory
Slew telescope to Park position.
Flushing events
Sending command
Scheduling check coroutines
process as completed...
[Telescope] delta Alt = +000.000 | delta Az= -000.001
Axes in position.
None
Cover state <MirrorCoverState.CLOSED: 6>
M1 cover already closed.
Close dome.
ATDome Shutter Door is already closed. Ignoring.
Slew dome to Park position.
Timed out trying to get azimuth in position from the dome just after command was sent. Continuing...
[Dome] delta Az = -069.220
[Dome] delta Az = -067.940
[Dome] delta Az = -066.430
[Dome] delta Az = -064.690
[Dome] delta Az = -062.720
[Dome] delta Az = -060.530
[Dome] delta Az = -058.100
[Dome] delta Az = -055.440
[Dome] delta Az = -052.560
[Dome] delta Az = -049.460
[Dome] delta Az = -046.140
[Dome] delta Az = -042.700
[Dome] delta Az = -039.270
[Dome] delta Az = -035.830
[Dome] delta Az = -032.390
[Dome] delta Az = -028.950
[Dome] delta Az = -025.520
[Dome] delta Az = -022.070
[Dome] delt

RuntimeError: Unable to put athexapod in STANDBY


In [ ]:
await attcs.stop_tracking()

In [ ]:
await attcs.atptg.cmd_stopTracking.start(timeout=5)

In [ ]:
await attcs.point_azel(target_name="Park position",
                                  az=attcs.tel_park_az,
                                  el=attcs.tel_park_el,
                                  wait_dome=False)

In [ ]:
valid_imagetype = ["BIAS", "DARK", "FLAT", "OBJECT", "ENGTEST"]
exptime = 10

In [ ]:
for imgtype in valid_imagetype:
    shutter = imgtype in ["BIAS", "DARK"]
    _exptime = exptime if imgtype != "BIAS" else 0.
    print(f"{imgtype} : shutter={shutter} : exptime={_exptime}")

In [11]:
attcs.check.atdome=True
await attcs.standby()

[atmcs]::[<State.STANDBY: 5>]
[atptg]::[<State.STANDBY: 5>]
[ataos]::[<State.STANDBY: 5>]
[atpneumatics]::[<State.STANDBY: 5>]
[athexapod]::[<State.STANDBY: 5>]
[atdome]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atdometrajectory]::[<State.STANDBY: 5>]
All components in standby.


In [14]:
await attcs.athexapod.cmd_start.start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1951810940, ack=<SalRetCode.CMD_TIMEOUT: -304>, error=1, result='Timeout')